<a href="https://colab.research.google.com/github/abhilasha-kumar/ALDA/blob/master/Copy_of_voter_turnout_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decoding Democracy workshop - PDF Scraper
<br>

We will be using this Jupyter notebook to execute the pdf scraping. These notebooks work by executing code one cell at a time. Inside each shaded grey box, there will be code. To run the code:
1. Hover over the brackets on the left of the cell [    ]  and a "play" button will appear. Clicking this button will run the code inside that cell. 
*Additionally, you can press Shift + Enter (Mac) or Ctrl + Enter (Windows/Linux) to run the cell*
<br>
2. While the code runs, the button will look like a square. If you need to quit running the code, press the button to stop execution.
<br>
3. Once the code is done, there will be a number inside the brackets [ 1 ]. This indicates the order in which the cells have been run. 
<br>
4. If the code creates any output, it will be displayed below the cell. For part of this code, it might be useful to interact with this output in full screen. To do so, select the menu with three dots on the right side of the code block and select "View output fullscreen".


## Section 1:
## Make a new directory and download the file

In [0]:
#This is a comment, it will not be run as part of the code's execution. Read these to understand what the code below it is doing!

#Run this code and make sure the output says "Hello World!"
print("Hello World!")

Hello World!


The next cell will make the directory, change to it and then display the information. Being able to invoke shell commands alongside Python code (or other languages). These commands are preceded by ! or %

### IMPORTANT: Change this directory name to the year you are working on!

In [0]:
# Make a new directory (mkdir)

## CHANGE THIS TO THE YEAR YOU ARE WORKING ON
!mkdir 2017

# Change to that directory (cd)

## CHANGE THIS AS WELL
%cd '2017'

# Make sure we have changed path by printing the working directory (pwd)
!pwd

/content/2017
/content/2017


The file path should look like * /content/YYYY*
<br><br>

---
<br>


Next, we will download the file. 

###IMPORTANT: Change the URL to reflect the file you are working on 
To do so: 

1.   Go to the [Github page](https://github.com/amanda-price/decoding-democracy) and navigate to the file you are working on. 
2.   Click on the file name.
3.   In the upper right hand corner, _right click_ the Download button and select "Copy Link Address". 
4.   Paste this address below after _wget_




In [0]:
# Change the url to the pdf you are interested in
## CHANGE THIS
!wget https://github.com/amanda-price/decoding-democracy/raw/master/2017/20170307_primary_municipal_precinct.pdf


#We will then display the list of files to make sure it downloaded
!ls

--2019-03-29 19:17:17--  https://github.com/amanda-price/decoding-democracy/raw/master/2017/20170307_primary_municipal_precinct.pdf
Resolving github.com (github.com)... 192.30.253.113, 192.30.253.112
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/amanda-price/decoding-democracy/master/2017/20170307_primary_municipal_precinct.pdf [following]
--2019-03-29 19:17:22--  https://raw.githubusercontent.com/amanda-price/decoding-democracy/master/2017/20170307_primary_municipal_precinct.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4593322 (4.4M) [application/octet-stream]
Saving to: ‘20170307_primary_municipal_precinct.pdf’

20170307_primary_mu 100%

<br>The file name should appear as the last line from the code above. If not, check the URL path and try again!

## Section 2
## Load all the necessary packages and functions

Run each cell in this block of codes before trying the main code below. This will install the packages and compile the functions needed. <br><br>
After each code block is run, you do not need to rerun them unless you make a change to them. 

In [0]:
# Install and load PDF scraper package
!pip install PyMuPDF

import fitz
from copy import deepcopy
import time
from math import ceil

    100% |████████████████████████████████| 5.4MB 6.5MB/s 


The message "Successfully installed PyMuPDF-1.14.12" should have been produced above. If not, there is an error installing the PDF scraping package.
<br><br>
The rest of the functions below will not produce an output. Make sure there is a number (i.e. [ 4 ] ) that appears after you run the code block. 

In [0]:
# define some functions to use later
def fetch_and_clean_text_from_page(document,page_index):
	# Returns a list where each item is a segment of text between 
	# two carriage returns and some basic data-entry mistakes have been corrected

	# load a page
	page = document.loadPage(page_index)
	# get the text on the page
	text = page.getText("text")
	#print(text)
  
	# replace possible stupid errors that occur throughout the document
	text=text.replace('Times\nCounted','Times Counted')
	text=text.replace('times\ncounted','Times Counted')
	text=text.replace('TIMES\nCOUNTED','Times Counted')
	text=text.replace('Times \nCounted','Times Counted')
	text=text.replace('times \ncounted','Times Counted')
	text=text.replace('TIMES \nCOUNTED','Times Counted')

	# remove the header which can be identified because the last line of the 
	# header is "Page: x of y" where y is the total number of pages in the 
	# document so the text "of y" is the same on every page
	text=text.split('of '+str(document.pageCount)+'\n')
  
	# split the text into data entries according to carriage returns
	text=text[1].split('\n')

	# remove "CITYWIDE" if it occurs
	if text[0].startswith('CITYWIDE') or text[0].startswith('CITY WIDE'):
		text=text[1:]
    
	# sometimes the last entry is empty because the text on the page ends with a carriage return
	if len(text[-1])==0:
		text=text[:-1]

	return text

In [0]:
def extract_column_labels_from_text(text_list):
  
	
	column_start_ndx=list(map(lambda x: len(x)>0 and (not x.startswith('    '))  and (not x[0].isdigit()) and (not '-'==x) , text)).index(True)
	column_stop_ndx=list(map(lambda x: len(x)>0 and (not x.startswith('    '))  and (x[0].isdigit() or '-'==x) , text)).index(True)
	column_labels=text[column_start_ndx:(column_stop_ndx-1)]
	
  # control for line breaks in column headers
	if any(list(map(lambda x: len(x)>0 and (x.endswith(' ')) , text))):
		column_labels2=[]
		skip_next=False
		for clmndx in range(len(column_labels)):
			if skip_next:
				skip_next=False
				continue
			if column_labels[clmndx].endswith(' ') and column_labels[clmndx].count(' ')<=2:
				skip_next=True
				column_labels2.append(column_labels[clmndx]+column_labels[clmndx+1])
			else:
				skip_next=False
				column_labels2.append(column_labels[clmndx])
		column_labels=deepcopy(column_labels2);
		column_labels2=[]
	
  # control for improperly split column headers
	if any(list(map(lambda x: len(x)>0 and (x.count(' ')>1 and x.count(' ')%2==1) , text))):
		column_labels2=[]
		for clmndx in range(len(column_labels)):
			if (column_labels[clmndx].count(' ')>1 and column_labels[clmndx].count(' ')%2==1):
				split_label=column_labels[clmndx].split(' ')
				for splitndx in range(0,len(split_label),2):
					column_labels2.append(split_label[splitndx]+' '+split_label[splitndx+1])
			else:
				column_labels2.append(column_labels[clmndx])
		column_labels=deepcopy(column_labels2);
		column_labels2=[]

	return column_labels

In [0]:
def identify_page_type(text_list,column_labels):
  
	column_stop_ndx=list(map(lambda x: len(x)>0 and (not x.startswith('    '))  and (x[0].isdigit() or '-'==x) , text)).index(True)
	ques_count=sum(list(map(lambda x: len(x)>0 and (x.startswith('Reg.')) , column_labels)))
	if ques_count==0:
		if 'TURN OUT'==text_list[column_stop_ndx-1].upper() or 'TURNOUT'==text_list[column_stop_ndx-1].upper() or 'TURN OUT'==text_list[column_stop_ndx].upper() or 'TURNOUT'==text_list[column_stop_ndx].upper():
			page_type='turnout'
			entries_in_column=[1 for i in range(len(column_labels))]
		else:
			page_type='overflow'
			entries_in_column=[2 for i in range(len(column_labels))]
	else:
		page_type='question'

	return page_type

In [0]:
def correct_columns_and_identify_number_of_entries_in_each_data_column(column_labels,page_type):
    
	ques_count=sum(list(map(lambda x: len(x)>0 and (x.startswith('Reg.')) , column_labels)))
	column_labels = checkColumnLabels(column_labels)

	if page_type=='turnout':        
		entries_in_column=[1 for i in range(len(column_labels))] 
	elif page_type=='overflow':
		entries_in_column=[2 for i in range(len(column_labels))]
	else:
		ques_start_ndcs=[i for i in range(len(column_labels)) if (len(column_labels[i])>0 and column_labels[i].startswith('Reg.'))]

		if ques_count==1 and ques_start_ndcs[0]==0:
			entries_in_column=[1 for i in range(3)]+[2 for i in range(len(column_labels)-3)]
		else:
			ques_start_ndcs2=[x for x in ques_start_ndcs if x!=0]
			column_labels2=[]
			entries_in_column=[]
			for clmndx in range(len(column_labels)):
				if column_labels[clmndx].startswith("Reg.") or column_labels[clmndx].startswith("Times Counted") or column_labels[clmndx].startswith("Total Votes"):
					entries_in_column.append(1)
				else:
					entries_in_column.append(2)

	return column_labels,entries_in_column

In [0]:
def checkColumnLabels(column_labels):
  
	global raw_column_labels
	global corrected_labels

	if raw_column_labels != column_labels:
		raw_column_labels = column_labels
		print("These are the column labels: {}".format(column_labels))
		print("Does this list match the column labels on page {}? Y or N".format(pageCount))
		response = input()
		if response.upper() == "Y":
			corrected_labels = column_labels
		else:
			corrected_labels = []
			for item in column_labels:
				print("Is this label correct Y or N: {}".format(item))
				answer = input()
				if answer.upper() =="Y":
					corrected_labels.append(item)
				else:
					print("Input the correct label. If this label should be deleted, enter DEL: ")
					new_label = input()
					if new_label.upper() == "DEL":
						continue
					else:
						corrected_labels.append(new_label.upper())
            
	return corrected_labels

In [0]:
def extract_row_labels(text_list):
  
  column_start_ndx=list(map(lambda x: len(x)>0 and (not x.startswith('    '))  and (not x[0].isdigit()) and (not '-'==x) , text)).index(True)
  row_labels=[x.replace('    ','') for x in text_list[0:column_start_ndx]]
  if 'TOTAL' in row_labels[-1].upper() and 'TOTAL' in row_labels[-2].upper():
    row_labels=row_labels[:-1]
    
  return row_labels
    
def extract_data_strings(text_list):
  
	column_stop_ndx=list(map(lambda x: len(x)>0 and (not x.startswith('    '))  and (x[0].isdigit() or '-'==x) , text)).index(True)
	data_strings=text_list[column_stop_ndx:]

	return data_strings

In [0]:
def check_for_errors_reading_page(data_strings,expected_size,entries_per_row):
  
	if len(data_strings)<expected_size:
		data_strings2=[]
		for dndx in range(len(data_strings)):
			data_strings2=data_strings2+deepcopy(data_strings[dndx].replace('  ',' ').split(' '))
		data_strings=deepcopy(data_strings2)
		data_strings2=[]
	success=True
	if len(data_strings)<expected_size:
		success=False
	if len(data_strings)>expected_size:
		temp=[];
		for tempndx in range(ceil(len(data_strings)/float(entries_per_row))):
			temp.append([data_strings.pop(0) for x in range(min([entries_per_row,len(data_strings)]))])
		print('size error in check_for_errors_reading_page')
		print(temp)
		success=False
    
	return success,data_strings

## Section 3
## Run the main program

This next section will execute the main part of the code. <br><br>
Make sure you change the *file* variable to save the files correctly. They will mimic the naming convention of the pdf you downloaded that has the date (YYYYMMDD format), general or primary, whether it is a municipal election and results granularity (precinct level). Such as *20190305_primary_municipal_precinct*
<br><br>
When you run the main code, you will need to check that the column labels it has extracted match the pdf. 

For example, the code may say:



> *These are the column labels: ['Write-In Votes', 'US REP', 'DIST 1', 'Reg. Voters', 'Times Counted', 'Total Votes', 'JAMILAH NASHEED', 'Write-In Votes']<br>
Does this candidate list match page 385? Y or N*



But the entries 'US REP' and 'DIST 1' is the office, not a column label. After entering 'N', you will have the option to remove these entries. 

If the number of column labels does not match the expected data size, there will be an error.

In [0]:
# enter the name for the input file without "_precinct.pdf"
## CHANGE THIS
file = "20170307_primary"

In [0]:
# open the document, check it and allocate the object to save in
global pageCount 
global raw_column_labels

doc = fitz.open(file+"_municipal_precinct.pdf")
data_frames_to_save=[];
current_data_frame={'Name':None,'columns':None,'index':[],'collection_label':None}
raw_column_labels = None

print("The document is {} pages long.".format(doc.pageCount))


for pndx in range(doc.pageCount):
  
	pageCount = int(pndx)+1
	# get the text from the page
	text=fetch_and_clean_text_from_page(doc,pndx)

	# tentatively identify the column labels
	column_labels=extract_column_labels_from_text(text) 

	for attempt in range(2):
    
		# identify whether this page shows turnout results, contains the start of a 
		# race/ballot-question or is an overflow for a race/ballot-question with more 
		# canditates/options that can fit on one page
		page_type=identify_page_type(text,column_labels)
    
		# Some columns have two data entries, we need to know this, if the page
		# introduces a new race/ballot-question but this does not occur on the first
		# collumn then we need to correct our column headers
		column_labels,entries_in_column=correct_columns_and_identify_number_of_entries_in_each_data_column(column_labels,page_type)

		# extract the labels for the rows
		row_labels=extract_row_labels(text)
    
		# extract the actual data
		data_strings=extract_data_strings(text)
    
		# we need to know how may data entries per row there are so that we can 
		# split the entries by number (since there is no row delimiter)
		entries_per_row=sum(entries_in_column) 
    
		# now we need to check that we extracted the data correctly by seeing if 
		# the number of data strings corresponds to the number we expected
		num_data_rows=(len(row_labels)-sum(list(map(lambda x: len(x)>0 and x.startswith('W ') , row_labels))))
		expected_size=entries_per_row*num_data_rows

		# now we identify the race/ballot-question
		# NOTE: if there are too many candidates/options than can fit on one page 
		# the label will only occur on the las page (in otherwords this is broken)
		column_stop_ndx=list(map(lambda x: len(x)>0 and (not x.startswith('    '))  and (x[0].isdigit() or '-'==x) , text)).index(True)
		collection_label=text[column_stop_ndx-1]
    
		# now we see if we read the data correctly, if we did then we don't need to 
		# run through a second time.
		success,data_strings=check_for_errors_reading_page(data_strings,expected_size,entries_per_row)
    
		if success:
			break
	  
	# if we have run through the page parsing section twice and still have not 
	# read things correctly then we have something to debug
	if not success:
		errorhere # a lazy way to make the code stop


	# now that we have parsed the page we need to store the data in a pandas ready dictionary
	empty_dict={} # this will become a blank template but it changes since each page has different collumn labels
  
	# first step is to create a dictionary where each key corresponds to a data 
	# column on the original page
	temp_data_dict={}
  
	for cndx in range(len(column_labels)):
		cndx2=sum(entries_in_column[:cndx])

		# since there are the same number of data strings per row then every Nth 
		# data_string belongs to the column (N=entries_per_row)
		# so we count off by N but need to adjust where we start counting according the column
		if entries_in_column[cndx]==1: # if this column has one entry per row
			temp_data_dict[column_labels[cndx]]=[data_strings[x] for x in range(cndx2,len(data_strings),entries_per_row)] # count by N and make a list of strings
		
		else: # if this column has two entries per row
			temp_data_dict[column_labels[cndx]]=list(zip([data_strings[x] for x in range(cndx2,len(data_strings),entries_per_row)],[data_strings[x] for x in range(cndx2+1,len(data_strings),entries_per_row)]))
		  # count by N but make a list of tuples where each tuple contains two strings
		
		empty_dict[column_labels[cndx]]=[]
		if num_data_rows!=len(temp_data_dict[column_labels[cndx]]): # if we split up the strings wrong then we have something to debug
			print('data size error 4')
			errorhere # a lazy way to make the code stop
	  
	# the object "current_data_frame" is initialized as empty but stores all the 
	# data iteratively until we have all the data for a polling place, even if it 
	# breaks accross pages. Once we have all the data for a polling place we start
	# with a new current_data_frame

	# initialize the current_data_frame if need be
	if current_data_frame['columns'] is None:
		current_data_frame['columns']=deepcopy(empty_dict)

	# now split the columns in temp_data_dict into sections corresponding to polling places
	current_ndx=0;
	# we step through removing data from temp_data_dict as we copy it into current_data_frame
	# we also copy and remove from row_labels which lets us know when we are done
	  
	while len(row_labels)>0:
		# get a logical indicating which row_labels correspond to the ID of a polling place
		next_polling_place_list=list(map(lambda x: len(x)>0 and x.startswith('W ') , row_labels));
		
		if any(next_polling_place_list): # if there are any more polling places
		  # get the index of where the next polling places data starts 
		  # (should be 5 unless a polling place breaks accross pages)
			next_ndx=next_polling_place_list.index(True) 
			next_Name=row_labels.pop(next_ndx); # get the name of the next polling place
		
		elif len(row_labels)>5 and "TOTAL" not in row_labels[-1].upper(): # if there are more row labels for a polling place then we expect then we have something to debug
			print('error')
			print(row_labels)
			errorhere # a lazy way to make the code stop
		
		else: # if there are no more polling places then just take all the remaining data
			next_ndx=len(row_labels)
			next_Name=None;

		# according to the panda framework the row labels are called "index"
		current_data_frame['index']=current_data_frame['index']+[row_labels.pop(current_ndx) for x in range(current_ndx,next_ndx)]
		
    # now we step through the columns in temp_data_dict and extract the data from that
		for k in current_data_frame['columns'].keys():
			current_data_frame['columns'][k]=current_data_frame['columns'][k]+[temp_data_dict[k].pop(current_ndx) for x in range(current_ndx,next_ndx)] # temp_data_dict[k].pop(pop_list))
		
		current_data_frame['collection_label']=collection_label
		if next_Name is not None: # this usually corresponds to city wide results
			data_frames_to_save.append(current_data_frame)
			current_data_frame={'Name':next_Name,'columns':deepcopy(empty_dict),'index':[]}
  
print("Section complete! Move on to the next section.")

The document is 672 pages long.
These are the column labels: ['Reg. Voters', 'Cards Cast', '% Turnout']
Does this list match the column labels on page 1? Y or N
Y
These are the column labels: ['Reg. Voters', 'Times Counted', 'Total Votes', 'ANDY KARANDZIEFF', 'JIM OSHER', 'ANDREW JONES']
Does this list match the column labels on page 25? Y or N
Y
These are the column labels: ['Reg. Voters', 'Times Counted', 'Total Votes', 'ANTONIO FRENCH', 'LEWIS REED', 'JEFFREY BOYD']
Does this list match the column labels on page 49? Y or N
Y
These are the column labels: ['TISHAURA JONES', 'LYDA KREWSON', 'WILLIAM HAAS', 'JIMMIE MATTHEWS']
Does this list match the column labels on page 73? Y or N
Y
These are the column labels: ['Reg. Voters', 'Times Counted', 'Total Votes', 'ROBB ', 'CUNNINGHAM', 'MAYOR - LIB', 'Reg. Voters', 'Times Counted', 'Total Votes', 'JOHNATHAN ', 'MCFARLAND']
Does this list match the column labels on page 97? Y or N
N
Is this label correct Y or N: Reg. Voters
Y
Is this label 

This scraper is not smart. It sometimes associates the wrong office to the candidate. Using the code below, you will be able to make sure that we have the correct office/candidate combination.<br>
Since there is no page number associated, it will be useful to use Ctrl+F to find the candidates name in the pdf. 

In [0]:

candidateList = []
officeCandidate = {}

for item in data_frames_to_save:
  
  if not item['collection_label'] == 'TURN OUT':
    
    for k in item['columns'].keys():
      
        if k.startswith("Reg. Voters") or k.startswith("Cards Cast") or k.startswith("% Turnout") or k.startswith("Times Counted") or k.startswith("YES") or k.startswith("NO") or k.startswith("Total Votes") or k.startswith("Write-In Votes"): 
          continue
        
        else:
          
          if k not in candidateList:
            candidateList.append(k)
            print("Is {} associated with the office {}? Y or N".format(k,item['collection_label']))
            answer = input()
            
            if answer.upper() == "Y":
              officeCandidate[k] = item['collection_label']
            
            else:
              print("Enter the office as seen on the record: ")
              response = input()
              officeCandidate[k] = response.upper()
            
print("Success. All candidates have been checked! If the list below is not correct, rerun this cell: ")

for k,v in officeCandidate.items():
  print("{} - {}".format(k,v))

Is ANDY KARANDZIEFF associated with the office MAYOR - REP? Y or N
Y
Is JIM OSHER associated with the office MAYOR - REP? Y or N
Y
Is ANDREW JONES associated with the office MAYOR - REP? Y or N
Y
Is ANTONIO FRENCH associated with the office MAYOR - DEM? Y or N
Y
Is LEWIS REED associated with the office MAYOR - DEM? Y or N
Y
Is JEFFREY BOYD associated with the office MAYOR - DEM? Y or N
Y
Is TISHAURA JONES associated with the office MAYOR - DEM? Y or N
Y
Is LYDA KREWSON associated with the office MAYOR - DEM? Y or N
Y
Is WILLIAM HAAS associated with the office MAYOR - DEM? Y or N
Y
Is JIMMIE MATTHEWS associated with the office MAYOR - DEM? Y or N
Y
Is ROBB CUNNINGHAM associated with the office MAYOR - GRN? Y or N
N
Enter the office as seen on the record: 
MAYOR - LIB
Is JOHNATHAN MCFARLAND associated with the office MAYOR - GRN? Y or N
Y
Is DARLENE GREEN associated with the office COMPTROLLER - DEM? Y or N
Y
Is ALEXANDRA JOHNSON associated with the office COMPTROLLER - DEM? Y or N
Y
Is 

Hooray, we have everything in order! Below we will write two files. The first file breaks down the detailed race information, including candidates. The second file will give us overall voter turnout per precinct. <br>
They are written into Comma-Separated Value (csv) files, which can be opened in Tableau, Excel or any basic text editor. 

In [0]:
# Write out the two files we want

#This file will hold the results for each race/candidate
output = open(file+"_results.csv","w")
output.write("ward,precinct,race,type,candidate,votes\n")

#This file will have overall turnout information
out2 = open(file+"_overall.csv",'w')
out2.write("ward,precinct,registered,polling,absentee,provisional,house,total\n")

for item in data_frames_to_save:
  if item['collection_label'] == 'TURN OUT':
    if not item['Name'] == None:
      ward = item['Name'][2:4]
      precinct = item['Name'][-2:]
      out2.write("{},{},{},{},{},{},{},{}\n".format(ward,precinct,item['columns']['Reg. Voters'][0],item['columns']['Cards Cast'][0],item['columns']['Cards Cast'][1],item['columns']['Cards Cast'][2],item['columns']['Cards Cast'][3],item['columns']['Cards Cast'][4]))
  else:
    for itemIndex in range(5):
      for k in item['columns'].keys():
        if k.startswith("Reg. Voters") or k.startswith("Cards Cast") or k.startswith("% Turnout") or k.startswith("Total Votes")or k.startswith("Times Counted") or k.startswith("Write-In Votes") or k.startswith("YES") or k.startswith("NO"): 
          continue
        else:
          num_votes = item['columns'][k][itemIndex][0]
          if not num_votes=='-':
            ward = item['Name'][2:4]
            precinct = item['Name'][-2:]
            output.write("{},{},{},{},{},{}\n".format(ward,precinct,officeCandidate[k],item['index'][itemIndex],k,num_votes))

print("Success!")
output.close()
out2.close()

Success!


The files should appear to the left under the tab Files. You may have to hit refresh _ON THE SIDEBAR_ (not your browser) in order to see them. 
You can quickly see if they look generally correct by double clicking the file. 
To download the file, right-click and choose Download.<br> <br>

### Before closing this code, please download these files. They will disappear when this instance is closed<br> <br>

After downloading, upload the copies to this [Google Drive folder](https://drive.google.com/drive/folders/1JNjUwriOtSPQsrDC-mqPbXb10tvi41hM?usp=sharing). Here you should also be able to access documents created by other groups. <br><br>

After that, it's on to visualization and analysis!
